In [0]:
spark

Out[17]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 local[*] 
 AppName 
 Databricks Shell

## Extração dos dados e transformações

In [0]:
casos_df = spark.read.table('casos_covid_opt_1_csv')
casos_df.createOrReplaceTempView('casos_view')
casos_df.show(5)

+-------+----+-------------+----------+---------+-----------+------------+-----------------+--------------------+
 regiao| uf| municipio| data|populacao|casos_novos|obitos_novos|recuperados_novos|acompanhamento_novos|
+-------+----+-------------+----------+---------+-----------+------------+-----------------+--------------------+
 Brasil|null| null|2020-02-25|210147125| 0| 0| 0| 0|
Sudeste| RJ|Volta Redonda|2020-12-31| 273012| 345| 8| 0| 0|
Sudeste| RJ|Volta Redonda|2020-12-30| 273012| 9| 9| 0| 0|
Sudeste| RJ|Volta Redonda|2020-12-29| 273012| 323| 6| 0| 0|
Sudeste| RJ|Volta Redonda|2020-12-28| 273012| 2| 0| 0| 0|
+-------+----+-------------+----------+---------+-----------+------------+-----------------+--------------------+
only showing top 5 rows

In [0]:
from pyspark.sql.functions import *

In [0]:
recuperados_acompanhamento_view = casos_df.groupBy('regiao', 'uf', 'municipio') \
    .agg(
        sum('recuperados_novos').alias('recuperados'),
        last('acompanhamento_novos').alias('em_acompanhamento'),
        last('data').alias('atualizacao')
    ) \
    .select('regiao', 'recuperados', 'em_acompanhamento') \
    .orderBy(col('municipio').asc_nulls_first(), 'regiao')

recuperados_acompanhamento_view.where('recuperados != 0 OR em_acompanhamento != 0').show()

+------+-----------+-----------------+
regiao|recuperados|em_acompanhamento|
+------+-----------+-----------------+
Brasil| 2920055795| 748883|
+------+-----------+-----------------+

In [0]:
# Acumulado | Incidência | Casos Novos
# Incidência por 100K habitantes Método de cálculo: (casos confirmados * 100.000) / população
# Fontes: https://www.coronavirus.sc.gov.br/sobre-os-dados/

casos_confirmados_view = casos_df.groupBy('regiao', 'uf', 'municipio') \
    .agg(
        sum('casos_novos').alias('casos_acumulados'),
        last('casos_novos').alias('casos_novos'),
        last('data').alias('atualizacao'),
        last('populacao').alias('agg_pop')
    ) \
    .withColumn('incidencia', round((col('casos_acumulados') * 100_000 ) / col('agg_pop'), 1)) \
    .select('regiao', 'uf', 'municipio', 'casos_acumulados', 'incidencia', 'casos_novos', 'atualizacao') \
    .orderBy(col('municipio').asc_nulls_first(), 'regiao')

casos_confirmados_view.show()

+------------+----+---------+----------------+----------+-----------+-----------+
 regiao| uf|municipio|casos_acumulados|incidencia|casos_novos|atualizacao|
+------------+----+---------+----------------+----------+-----------+-----------+
 Brasil|null| null| 18855015| 8972.3| 24605| 2021-01-01|
Centro-Oeste| GO| null| 686433| 9780.5| 244| 2021-01-01|
Centro-Oeste| MT| null| 456155| 13091.1| 1374| 2021-06-30|
Centro-Oeste| MS| null| 339323| 12210.3| 676| 2021-07-01|
Centro-Oeste| DF| null| 434708| 14416.9| 690| 2021-07-01|
 Nordeste| RN| null| 347248| 9902.0| 270| 2021-01-01|
 Nordeste| PE| null| 561505| null| 1| 2020-08-01|
 Nordeste| PI| null| 299084| 9137.3| 981| 2020-08-01|
 Nordeste| AL| null| 220793| 6615.8| 516| 2021-06-30|
 Nordeste| PB| null| 402175| 10009.0| 462| 2021-07-01|
 Nordeste| CE| null| 905258| 9912.9| 1794| 2021-07-01|
 Nordeste| SE| null| 266590| 11597.4| 701| 2021-07-01|
 Nordeste| BA| null| 1152476| 7748.7| 1683| 2021-06-30|
 Nordeste| MA| null| 322052| 4551.9| 21| 2021-01-01|
 Norte| AM| null| 405066| 9773.4| 561| 2021-01-01|
 Norte| AC| null| 85997| 9750.9| 180| 2021-07-01|
 Norte| AP| null| 118066| 13960.2| 197| 2021-06-30|
 Norte| RR| null| 116376| 19211.5| 21| 2021-01-01|
 Norte| PA| null| 557708| 6482.8| 1368| 2020-08-01|
 Norte| TO| null| 200243| null| 272| 2021-06-30|
+------------+----+---------+----------------+----------+-----------+-----------+
only showing top 20 rows

In [0]:
# Obitos Acumulados | Casos Novos | Letalidade | mortalidade
# Letalidade - Método de cálculo: (número de óbitos x 100) / número de casos confirmados
# Mortalidade -  Método de cálculo: (óbitos * 100.000) / população  -> por 100K habitantes

obitos_confirmados_view = casos_df.groupBy('regiao', 'uf', 'municipio') \
    .agg(
        sum('obitos_novos').alias('obitos_acumulados'),
        last('obitos_novos').alias('casos_novos'),
        sum('casos_novos').alias('casos_confirmados'),
        last('populacao').alias('agg_pop'),
        last('data').alias('atualizacao')
    ) \
    .withColumn('letalidade', round(((col('obitos_acumulados') * 100) / col('casos_confirmados')), 1)) \
    .withColumn('mortalidade', round((col('obitos_acumulados') * 100_000) / col('agg_pop'), 1)) \
    .select('regiao', 'uf', 'municipio', 'obitos_acumulados', 'casos_novos', 'letalidade', 'mortalidade', 'atualizacao') \
    .orderBy(col('municipio').asc_nulls_first(), 'regiao')

obitos_confirmados_view.show()

+------------+----+---------+-----------------+-----------+----------+-----------+-----------+
 regiao| uf|municipio|obitos_acumulados|casos_novos|letalidade|mortalidade|atualizacao|
+------------+----+---------+-----------------+-----------+----------+-----------+-----------+
 Brasil|null| null| 526892| 462| 2.8| 250.7| 2021-01-01|
Centro-Oeste| GO| null| 19485| 0| 2.8| 277.6| 2021-01-01|
Centro-Oeste| MT| null| 12000| 25| 2.6| 344.4| 2021-06-30|
Centro-Oeste| MS| null| 8400| 27| 2.5| 302.3| 2021-07-01|
Centro-Oeste| DF| null| 9322| 13| 2.1| 309.2| 2021-07-01|
 Nordeste| RN| null| 6853| 0| 2.0| 195.4| 2021-01-01|
 Nordeste| PE| null| 17953| 0| 3.2| null| 2020-08-01|
 Nordeste| PI| null| 6662| 25| 2.2| 203.5| 2020-08-01|
 Nordeste| AL| null| 5450| 20| 2.5| 163.3| 2021-06-30|
 Nordeste| PB| null| 8724| 22| 2.2| 217.1| 2021-07-01|
 Nordeste| CE| null| 22791| 56| 2.5| 249.6| 2021-07-01|
 Nordeste| SE| null| 5773| 16| 2.2| 251.1| 2021-07-01|
 Nordeste| BA| null| 24692| 106| 2.1| 166.0| 2021-06-30|
 Nordeste| MA| null| 9190| 6| 2.9| 129.9| 2021-01-01|
 Norte| AM| null| 13349| 10| 3.3| 322.1| 2021-01-01|
 Norte| AC| null| 1760| 3| 2.0| 199.6| 2021-07-01|
 Norte| AP| null| 1857| 2| 1.6| 219.6| 2021-06-30|
 Norte| RR| null| 1778| 6| 1.5| 293.5| 2021-01-01|
 Norte| PA| null| 15624| 22| 2.8| 181.6| 2020-08-01|
 Norte| TO| null| 3266| 6| 1.6| null| 2021-06-30|
+------------+----+---------+-----------------+-----------+----------+-----------+-----------+
only showing top 20 rows

## Carregamento dos dados

In [0]:
# Salvando a Primeira view como tabela hive:
recuperados_acompanhamento_view.write.saveAsTable('recuperados_acompanhamento')

In [0]:
# Salvando a Segunda view em formato parquet e compressao snappy
casos_confirmados_view.write.option('compression', 'snappy').parquet('/user/root/data')

In [0]:
# # Salvando a terceira View como um tópico Kafka
BROKERS_SERVERS = 'localhost:9092'
TOPICS = 'obitos_view'

obitos_confirmados_view.selectExpr('to_json(struct(*)) AS value') \
    .write \
    .format('kafka') \
    .option('kafka.bootstrap.servers', BROKERS_SERVERS)
    .option('topic', TOPICS) \
    .save()
    

## Main View

In [0]:
# Casos | Óbitos | Indicencia 100k/hab | Mortalidade | Atualização
# Incidência por 100K habitantes Método de cálculo: (casos confirmados * 100.000) / população
# Mortalidade -  Método de cálculo: (óbitos * 100.000) / população  -> por 100K habitantes

casos = casos_df.groupBy('regiao', 'municipio') \
    .agg(
        sum('casos_novos').alias('casos'),
        sum('obitos_novos').alias('obitos'),
        last('data').alias('atualizacao'),
        last('populacao').alias('agg_pop'),
    ) \
    .withColumn('incidencia', round((col('casos') * 100_000) / col('agg_pop'), 1)) \
    .withColumn('mortalidade', round((col('obitos') * 100_000) / col('agg_pop'), 1)) \
    .select('regiao', 'municipio', 'casos', 'obitos', 'incidencia', 'mortalidade', 'atualizacao') \
    .orderBy(col('municipio').asc_nulls_first(), 'regiao')

casos.show()

+------------+-------------------+--------+------+----------+-----------+-----------+
 regiao| municipio| casos|obitos|incidencia|mortalidade|atualizacao|
+------------+-------------------+--------+------+----------+-----------+-----------+
 Brasil| null|18855015|526892| 8972.3| 250.7| 2021-01-01|
Centro-Oeste| null| 1916619| 49207| 68968.3| 1770.7| 2021-07-01|
 Nordeste| null| 4477181|108088| 49027.0| 1183.6| 2021-07-01|
 Norte| null| 1735433| 43860| 196775.6| 4973.2| 2021-07-01|
 Sudeste| null| 7148156|245851| null| null| 2021-01-01|
 Sul| null| 3617963| 80879| null| null| 2021-01-01|
Centro-Oeste| Abadia de Goiás| 1508| 34| 17189.1| 387.6| 2021-07-01|
 Sudeste|Abadia dos Dourados| 444| 14| 6352.8| 200.3| 2021-07-01|
Centro-Oeste| Abadiânia| 443| 34| 2210.4| 169.6| 2021-07-01|
 Norte| Abaetetuba| 8894| 219| 5639.9| 138.9| 2021-01-01|
 Sudeste| Abaeté| 1313| 29| 5650.5| 124.8| 2021-07-01|
 Nordeste| Abaiara| 586| 15| 4992.8| 127.8| 2021-07-01|
 Nordeste| Abaré| 971| 11| 4834.2| 54.8| 2021-06-30|
 Sul| Abatiá| 1043| 18| 13986.9| 241.4| 2021-01-01|
 Nordeste| Abaíra| 288| 12| 3295.6| 137.3| 2021-06-30|
 Sul| Abdon Batista| 419| 4| 16348.0| 156.1| 2020-12-31|
 Norte| Abel Figueiredo| 970| 10| 13048.2| 134.5| 2021-01-01|
 Sul| Abelardo Luz| 1529| 55| 8540.0| 307.2| 2020-12-31|
 Sudeste| Abre Campo| 1171| 15| 8703.7| 111.5| 2021-07-01|
 Nordeste| Abreu e Lima| 3185| 230| 3185.3| 230.0| 2020-08-01|
+------------+-------------------+--------+------+----------+-----------+-----------+
only showing top 20 rows

## Sending Data to ElasticSearch

In [0]:
casos.write \
    .format('org.elasticsearch.spark.sql') \
    .option('es.port', ELASTIC_PORT) \
    .option('es.nodes', ELASTIC_ENDPOINT) \
    .mode('append') \
    .save('casos/_doc')